# Deutsch-Jozsa Algorithm #

In [ ]:
# imports
import os;
import sys;

# NOTE: need this to force jupyter to reload imports:
for key in list(sys.modules.keys()):
    if key.startswith('src.'):
        del sys.modules[key];

os.chdir(os.path.dirname(_dh[0]));
sys.path.insert(0, os.getcwd());

from src.thirdparty.maths import *;
from src.thirdparty.quantum import *;
from src.demo.deutsch_jozsa import *;

np.random.seed(39102901); # for repeatability

In [ ]:
action_prepare_circuit_and_job(
    option = BACKEND_SIMULATOR.AER,
    n = 4,
    num_shots = 10000,
);
action_display_statistics(queue=False);

In [ ]:
# Run with cloud computing
action_prepare_circuit_and_job(
    option = BACKEND.LEAST_BUSY,
    n = 4,
    num_shots = 20000,
);

In [ ]:
action_display_statistics(queue=True);